# 02 - Feature Engineering with Feast & Ray

This notebook demonstrates **Feast Feature Store** with **Ray** for distributed processing:

1. **Generate sample data** - Create sales, store, and department data
2. **Configure Feast** - Set up feature store with Ray offline store
3. **Register features** - Define entities and feature views
4. **Historical retrieval** - Fetch features using Ray (distributed)
5. **Materialize** - Push features to online store for real-time serving

## Integration: Feast + KubeRay

```
┌─────────────────────────────────────────────────────────────────┐
│                         Feast + Ray                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   ┌──────────┐       ┌───────────────────────────────────┐      │
│   │ Notebook │──────▶│         KubeRay Cluster           │      │
│   │(Feast SDK)│      │  ┌────────┐ ┌────────┐ ┌────────┐ │      │
│   └──────────┘       │  │ Worker │ │ Worker │ │ Worker │ │      │
│        │             │  └────────┘ └────────┘ └────────┘ │      │
│        │             └───────────────────────────────────┘      │
│        │                           │                            │
│        ▼                           ▼                            │
│   ┌──────────┐              ┌──────────────┐                   │
│   │PostgreSQL│◀─────────────│ Parquet Data │                   │
│   │ Registry │              │ (Distributed)│                   │
│   └──────────┘              └──────────────┘                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```


In [ ]:
# Install dependencies
%pip install -q feast[postgres,ray] pandas pyarrow numpy


In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
from pathlib import Path

# Configuration
NAMESPACE = os.getenv("NAMESPACE", "feast-mlops-demo")
POSTGRES_HOST = os.getenv("POSTGRES_HOST", f"postgres.{NAMESPACE}.svc.cluster.local")
RAY_HOST = os.getenv("RAY_HOST", f"feast-ray-head-svc.{NAMESPACE}.svc.cluster.local")

# Paths
DATA_DIR = "/shared/data"
FEATURE_REPO_DIR = "/shared/feature_repo"

print(f"📌 Namespace: {NAMESPACE}")
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Feature repo: {FEATURE_REPO_DIR}")


## 1. Generate Sample Data

Creating realistic sales data for 10 stores, 5 departments over 2 years.


In [ ]:
def generate_sales_data(num_stores=10, num_depts=5, weeks=104):
    """Generate realistic sales data"""
    np.random.seed(42)
    
    records = []
    base_date = datetime(2022, 1, 1, tzinfo=timezone.utc)
    
    for week in range(weeks):
        date = base_date + timedelta(weeks=week)
        
        for store_id in range(1, num_stores + 1):
            store_base = 50000 + store_id * 5000
            
            for dept_id in range(1, num_depts + 1):
                # Base sales with seasonal pattern
                seasonal = 1 + 0.3 * np.sin(2 * np.pi * week / 52)
                # Holiday boost (week 47-52)
                holiday = 1.5 if 47 <= (week % 52) <= 52 else 1.0
                # Department factor
                dept_factor = 0.5 + dept_id * 0.2
                
                weekly_sales = (
                    store_base * dept_factor * seasonal * holiday
                    + np.random.normal(0, 2000)
                )
                
                records.append({
                    "store_id": store_id,
                    "dept_id": dept_id,
                    "event_timestamp": date,
                    "weekly_sales": max(0, weekly_sales),
                    "is_holiday": int(holiday > 1),
                    "temperature": 60 + 20 * np.sin(2 * np.pi * week / 52) + np.random.normal(0, 5),
                    "fuel_price": 3.0 + 0.5 * np.random.random(),
                    "cpi": 220 + week * 0.1,
                    "unemployment": 5.0 + np.random.normal(0, 0.5)
                })
    
    return pd.DataFrame(records)

# Generate data
print("Generating sales data...")
sales_df = generate_sales_data()
print(f"✅ Generated {len(sales_df):,} records")
print(f"   Date range: {sales_df['event_timestamp'].min()} to {sales_df['event_timestamp'].max()}")
sales_df.head()


In [ ]:
# Generate store features
def generate_store_features(num_stores=10):
    """Generate store metadata"""
    np.random.seed(42)
    base_date = datetime(2022, 1, 1, tzinfo=timezone.utc)
    
    records = []
    store_types = ["A", "B", "C"]
    
    for store_id in range(1, num_stores + 1):
        records.append({
            "store_id": store_id,
            "event_timestamp": base_date,
            "store_type": store_types[store_id % 3],
            "store_size": 100000 + store_id * 10000,
            "region": f"region_{(store_id - 1) // 3 + 1}"
        })
    
    return pd.DataFrame(records)

store_df = generate_store_features()
print(f"✅ Generated store features: {len(store_df)} stores")
store_df.head()


In [ ]:
# Save to parquet
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)

sales_path = f"{DATA_DIR}/sales_features.parquet"
store_path = f"{DATA_DIR}/store_features.parquet"

sales_df.to_parquet(sales_path)
store_df.to_parquet(store_path)

print(f"✅ Saved: {sales_path}")
print(f"✅ Saved: {store_path}")


## 2. Configure Feast Feature Store

Setting up Feast with:
- **PostgreSQL** registry for metadata
- **Ray** offline store for distributed feature retrieval
- **PostgreSQL** online store for real-time serving


In [ ]:
# Create feature_store.yaml
Path(FEATURE_REPO_DIR).mkdir(parents=True, exist_ok=True)

feature_store_yaml = f"""
project: sales_forecasting
provider: local

registry:
  registry_type: sql
  path: postgresql+psycopg2://feast:feast123@{POSTGRES_HOST}:5432/feast

offline_store:
  type: file  # Use file for simplicity, Ray for production scale

online_store:
  type: postgres
  host: {POSTGRES_HOST}
  port: 5432
  database: feast
  user: feast
  password: feast123

entity_key_serialization_version: 2
"""

with open(f"{FEATURE_REPO_DIR}/feature_store.yaml", "w") as f:
    f.write(feature_store_yaml)

print(f"✅ Created: {FEATURE_REPO_DIR}/feature_store.yaml")
print(feature_store_yaml)


## 3. Define Feature Views

Register entities and feature views in Feast.


In [ ]:
from feast import FeatureStore, Entity, FeatureView, Field, FileSource
from feast.types import Float32, Int64, String
from datetime import timedelta

# Initialize feature store
store = FeatureStore(repo_path=FEATURE_REPO_DIR)

# Define entities
store_entity = Entity(name="store_id", join_keys=["store_id"])
dept_entity = Entity(name="dept_id", join_keys=["dept_id"])

# Define data sources
sales_source = FileSource(
    path=f"{DATA_DIR}/sales_features.parquet",
    timestamp_field="event_timestamp",
)

store_source = FileSource(
    path=f"{DATA_DIR}/store_features.parquet",
    timestamp_field="event_timestamp",
)

# Define feature views
sales_features = FeatureView(
    name="sales_features",
    entities=[store_entity, dept_entity],
    ttl=timedelta(days=365),
    schema=[
        Field(name="weekly_sales", dtype=Float32),
        Field(name="is_holiday", dtype=Int64),
        Field(name="temperature", dtype=Float32),
        Field(name="fuel_price", dtype=Float32),
        Field(name="cpi", dtype=Float32),
        Field(name="unemployment", dtype=Float32),
    ],
    source=sales_source,
)

store_features = FeatureView(
    name="store_features",
    entities=[store_entity],
    ttl=timedelta(days=365),
    schema=[
        Field(name="store_type", dtype=String),
        Field(name="store_size", dtype=Int64),
        Field(name="region", dtype=String),
    ],
    source=store_source,
)

print("✅ Feature views defined")


In [ ]:
# Apply feature definitions to registry
store.apply([store_entity, dept_entity, sales_features, store_features])

print("✅ Features registered in Feast!")
print(f"\nRegistered feature views:")
for fv in store.list_feature_views():
    print(f"  - {fv.name}: {len(fv.features)} features")


## 4. Historical Feature Retrieval

Fetch training features using point-in-time joins.


In [ ]:
# Create entity dataframe for feature retrieval
# Sample recent records for training
entity_df = sales_df[["store_id", "dept_id", "event_timestamp"]].copy()
entity_df = entity_df.sample(n=min(5000, len(entity_df)), random_state=42)

print(f"📊 Entity dataframe: {len(entity_df)} rows")
entity_df.head()


In [ ]:
%%time
# Fetch historical features
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "sales_features:weekly_sales",
        "sales_features:is_holiday",
        "sales_features:temperature",
        "sales_features:fuel_price",
        "sales_features:cpi",
        "sales_features:unemployment",
        "store_features:store_type",
        "store_features:store_size",
    ]
).to_df()

print(f"✅ Retrieved {len(training_df)} training samples")
print(f"   Columns: {list(training_df.columns)}")
training_df.head()


In [ ]:
# Save training data for model training
training_path = f"{DATA_DIR}/training_data.parquet"
training_df.to_parquet(training_path)
print(f"✅ Saved training data: {training_path}")


## 5. Materialize to Online Store

Push features to PostgreSQL online store for real-time serving.


In [ ]:
%%time
# Materialize features to online store
from datetime import datetime, timezone

start_date = datetime(2022, 1, 1, tzinfo=timezone.utc)
end_date = datetime(2024, 1, 1, tzinfo=timezone.utc)

print("Materializing features to online store...")
store.materialize(start_date=start_date, end_date=end_date)
print("✅ Materialization complete!")
